In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from scipy.stats import ttest_ind
import numpy as np
from IPython.display import display, Markdown, Latex
plt.style.use('ggplot')
from datetime import date
from scipy.stats import norm
import numpy as np
import pandas

data_path = '/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/'

# Update mobility variables

In [6]:
#income and age added in census folder notebook before continuing here
mob_vars = pd.read_csv(data_path + 'mobility_heat/mob_vars_final_clean_90_v1.csv')
#relative_heat = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/heatwave/relative_heat_scores.csv')

In [24]:
mob_vars.groupby('year')['parks_difference'].describe()

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2020,623.0,-0.101124,26.833314,-105.0,-14.0,-5.0,7.5,178.0
2021,1091.0,-2.372136,34.689407,-209.0,-18.0,-6.0,7.0,320.0
2022,1127.0,0.850932,28.808410,-124.0,-15.0,-4.0,11.0,165.0


In [3]:
mob_vars['date'] = pd.to_datetime(mob_vars['date'])

In [4]:
mob_vars['month'] = [x.month for x in mob_vars['date']]

### Add winter temperatures

In [5]:
winter = pd.read_csv('/Users/shivyucel/Documents/projects/DPhil/Code_Data/data/winter_v2.csv')

In [6]:
mob_vars = mob_vars.merge(winter[['location', 'temperature_2m_winter']])

### Build >=5 duration variable

In [7]:
mob_vars['heatwave_dur'] = ['<5 days' if x<5 else '>=5 days' for x in mob_vars['Heat Wave Duration']]
mob_vars['hw_dur_>=_5'] = [0 if x=='<5 days' else 1 for x in mob_vars['heatwave_dur']]

### Get stage_frac variables

In [8]:
mob_vars['stage_frac'] = mob_vars['stage'] / mob_vars['Heat Wave Duration'] * 100


In [9]:
mob_vars['stage_frac'].describe()


count    6100.000000
mean       63.878364
std        28.344134
min         9.090909
25%        40.000000
50%        62.500000
75%       100.000000
max       100.000000
Name: stage_frac, dtype: float64

In [10]:
mob_vars.to_csv(data_path + 'mobility_heat/final_premultivariate_clean_90_v1.csv')

# Add/prepare final variables multivariate


In [11]:
mob_vars = pd.read_csv(data_path + 'mobility_heat/final_premultivariate_clean_90_v1.csv')

## Merge coastal 

In [12]:
coast_variable = pd.read_csv(data_path + 'mobility_heat/multivariate_modelling_v25.csv')
coast_variable.drop_duplicates('location', inplace=True)

In [13]:
mob_vars = mob_vars.merge(coast_variable[['location', 'coast_binary']], on='location')

In [14]:
df = mob_vars.copy()

### Remove Outliers

In [ ]:
SD = 4

In [15]:
df['workplaces_difference'] = [np.nan if (x >= df['workplaces_difference'].mean() + (SD * df['workplaces_difference'].std())) | (x <= df['workplaces_difference'].mean() - (SD * df['workplaces_difference'].std())) else x for x in df['workplaces_difference'] ]

df['residential_difference'] = [np.nan if (x >= df['residential_difference'].mean() + (SD * df['residential_difference'].std())) | (x <= df['residential_difference'].mean() - (SD * df['residential_difference'].std())) else x for x in df['residential_difference'] ]


df['grocery_pharmacy_difference'] = [np.nan if (x >= df['grocery_pharmacy_difference'].mean() + (SD * df['grocery_pharmacy_difference'].std())) | (x <= df['grocery_pharmacy_difference'].mean() - (SD * df['grocery_pharmacy_difference'].std())) else x for x in df['grocery_pharmacy_difference'] ]

df['parks_difference'] = [np.nan if (x >= df['parks_difference'].mean() + (SD * df['parks_difference'].std())) | (x <= df['parks_difference'].mean() - (SD * df['parks_difference'].std())) else x for x in df['parks_difference'] ]

df['retail_difference'] = [np.nan if (x >= df['retail_difference'].mean() + (SD * df['retail_difference'].std())) | (x <= df['retail_difference'].mean() - (SD * df['retail_difference'].std())) else x for x in df['retail_difference'] ]


df['transit_difference'] = [np.nan if (x >= df['transit_difference'].mean() + (SD * df['transit_difference'].std())) | (x <= df['transit_difference'].mean() - (SD * df['transit_difference'].std())) else x for x in df['transit_difference'] ]


df.dropna(how='all', subset=['workplace_differences', 'residential_difference', 'transit_difference', 'grocery_pharm_difference', 'parks_difference', 'retail_rec_difference'], inplace=True)

In [33]:
df[['workplaces_difference', 'residential_difference', 'transit_difference', 'grocery_pharmacy_difference', 'parks_difference', 'retail_difference']].describe()

,workplaces_difference,residential_difference,transit_difference,grocery_pharmacy_difference,parks_difference,retail_difference
count,5941.000000,4884.000000,3391.000000,4482.000000,2818.000000,5227.000000
mean,-0.518431,-0.077600,-0.680035,1.242972,-1.605749,0.426057
std,3.566710,1.501967,8.479401,6.326961,26.826260,6.721429
min,-17.000000,-6.000000,-41.000000,-27.000000,-119.000000,-31.000000
25%,-2.000000,-1.000000,-4.000000,-2.000000,-16.000000,-3.000000
50%,0.000000,0.000000,-1.000000,1.000000,-5.000000,0.000000
75%,1.000000,1.000000,3.000000,4.000000,8.000000,3.000000
max,16.000000,6.000000,41.000000,30.000000,122.000000,31.000000


### Center heat wave temperature variable

In [16]:
df['heat_temp_centered'] = df['temperature_2m_heat'] - df['temperature_2m_heat'].mean()


### Get Large city Variables

In [17]:
df['large_city'] = [x if x in ['Los Angeles County', 'San Diego County', 'Santa Clara County', 'San Francisco County', 'Sacramento County', 'King County', 'Multnomah County', 'Greater Vancouver'] else 'nonmajor' for x  in df['county']]

### Get coast binary as digit

In [18]:
df['coast_binary'] = df['coast_binary'] * 1

### Get June heatwave variable

In [19]:
df = df[['temperature_2m_heat', 'month', 'date_heat', 'control_day', 'Heat Wave Duration', 'large_city', 'date', 'location', 'heat_ID_x', 'workplaces_difference', 'residential_difference', 'grocery_pharmacy_difference', 'retail_difference', 'parks_difference', 'transit_difference', 'state', 'year', 'weekend', 'stage_frac', 'heatwave_dur', 'temp_diff', 'heat_temp_centered', 'coast_binary', 'temperature_2m_winter', 'income_dec', 'over65_dec', 'nonwhite_dec','large_city', 'popdens_dec']]

In [20]:
df['index'] = range(0, len(df))

In [21]:
df['date'] = pd.to_datetime(df['date'])

In [22]:
df['month'] = [x.month for x in df['date']]

In [23]:
df['month'].unique()

array([6, 7, 8, 9])

In [24]:
df['june_hw'] = df.groupby(['heat_ID_x'])['month'].transform(lambda x: 6 in x.unique())
df['june_hw'] = df['june_hw'] * 1



In [25]:
df['month'].value_counts()

month
8    2347
7    2189
9     796
6     754
Name: count, dtype: int64

In [26]:
df.rename(columns={'heat_ID_x': 'heat_ID'}, inplace=True)

In [27]:
df.reset_index(inplace=True, drop=True)

In [28]:
df['temperature_2m_heat'].describe()

count    6086.000000
mean       33.392871
std         5.037908
min        18.414703
25%        29.775768
50%        33.688816
75%        36.818149
max        47.254585
Name: temperature_2m_heat, dtype: float64

In [ ]:
df.to_csv(data_path + 'mobility_heat/mobility_heat_dataset_update.csv')